In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline

In [2]:
df=pd.read_csv('train.csv')
df_copy=df[['cat_var_1','cat_var_2', 'cat_var_3', 'cat_var_6', 'cat_var_8', 'cat_var_9','cat_var_10', 'cat_var_13', 'cat_var_15']]
df2=pd.read_csv('test.csv')
df2_copy=df2[['cat_var_1','cat_var_2', 'cat_var_3', 'cat_var_6', 'cat_var_8', 'cat_var_9','cat_var_10', 'cat_var_13', 'cat_var_15']]

In [3]:
x=df.corr()
y_train=df['target']

In [4]:
ls=[]
ls=x.columns
j=0
for i in x['target']:
    if str(i)=='nan':
        x.drop(ls[j], inplace=True)
        j=j+1
        i=i+1
    else:
        j=j+1
        i=i+1

In [5]:
cols=[]
rem=[]
cols=x.reset_index()['index']
rem=cols

In [6]:
j=0
for i in x['target']:
    if abs(i) < 0.01:
        x.drop(rem[j], inplace=True)
        i=i+1
        j=j+1
    else:
        i=i+1
        j=j+1    

In [7]:
cols=[]
cols=x.reset_index()['index']
cols=cols[:-1]

In [8]:
heads=[]
heads=df.columns[1:df.columns.shape[0]-1]
discard=[]
discard=list(set(cols).symmetric_difference(heads))

In [9]:
df.drop(discard, axis=1, inplace=True)
df2.drop(discard, axis=1, inplace=True)
df.drop(['transaction_id', 'target'], axis=1, inplace=True)
df2.drop(['transaction_id'], axis=1, inplace=True)

In [10]:


from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

df_copy = DataFrameImputer().fit_transform(df_copy)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for feature in df_copy.columns:
    if df_copy[feature].dtype == 'object':
        df_copy[feature] = le.fit_transform(df_copy[feature])

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(random_state=0)
clf = clf.fit(df_copy, y_train)
clf.feature_importances_  

model = SelectFromModel(clf, prefit=True)

X_train_opt = model.transform(df_copy)
feature_idx_train = model.get_support()
feature_name_train = df_copy.columns[feature_idx_train]

print("Important categorical features are: \n")
print(feature_name_train)

Important categorical features are: 

Index(['cat_var_3', 'cat_var_10', 'cat_var_15'], dtype='object')


In [11]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(random_state=0)
clf = clf.fit(df, y_train)
clf.feature_importances_  

model = SelectFromModel(clf, prefit=True)

In [12]:
X_train_opt = model.transform(df)
feature_idx_train = model.get_support()
feature_name_train = df.columns[feature_idx_train]

In [13]:
print("Important features are: \n")
print(feature_name_train)

Important features are: 

Index(['num_var_1', 'num_var_2', 'num_var_4', 'num_var_5', 'num_var_6',
       'num_var_7'],
      dtype='object')


In [14]:
df2_copy.drop(['cat_var_1', 'cat_var_2','cat_var_6', 'cat_var_8',
       'cat_var_9', 'cat_var_13'], inplace=True, axis=1)
df_copy.drop(['cat_var_1', 'cat_var_2','cat_var_6', 'cat_var_8',
       'cat_var_9', 'cat_var_13'], inplace=True, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [15]:

keep=['cat_var_3', 'cat_var_10', 'cat_var_15', 'num_var_1', 'num_var_2', 'num_var_4', 'num_var_6',
       'num_var_7' ]
print(keep)

['cat_var_3', 'cat_var_10', 'cat_var_15', 'num_var_1', 'num_var_2', 'num_var_4', 'num_var_6', 'num_var_7']


In [16]:
df=pd.read_csv('train.csv')
df2=pd.read_csv('test.csv')

transaction_id=df2['transaction_id']
y_train=df['target']

df=df[['cat_var_3', 'cat_var_10', 'cat_var_15', 'num_var_1', 'num_var_2', 'num_var_4', 'num_var_6', 'num_var_7']]
df2=df2[['cat_var_3', 'cat_var_10', 'cat_var_15', 'num_var_1', 'num_var_2', 'num_var_4', 'num_var_6', 'num_var_7']]

In [17]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

all_X = df.append(df2)
all_X_imputed = DataFrameImputer().fit_transform(all_X)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for feature in all_X_imputed.columns:
    if all_X_imputed[feature].dtype == 'object':
        all_X_imputed[feature] = le.fit_transform(all_X_imputed[feature])

X_train = all_X_imputed[0:df.shape[0]]
X_test = all_X_imputed[df.shape[0]::]


In [18]:
from xgboost import XGBClassifier
model=XGBClassifier()

model.fit(X_train, y_train)

y_pred=model.predict(X_test)
predi=model.predict_proba(X_test)

In [19]:
to_insert=pd.DataFrame(predi, columns=['ignore','accept'])
prediction=pd.DataFrame(columns=['transaction_id','target'])
prediction['transaction_id']=transaction_id
prediction['target']=to_insert['accept']
prediction.to_csv('bw_xgboost.csv', index=False)